In [ ]:
# Import netket library
import netket as nk
from qiskit.chemistry import FermionicOperator
# Helper libraries
import numpy as np
import itertools
import matplotlib.pyplot as plt

In [16]:
#H2
OB = np.load('../data/integrals/STO-3G/STO-3G_H2_OB_d0-7339_eq1.npy')
TB = np.load('../data/integrals/STO-3G/STO-3G_H2_TB_d0-7339_eq1.npy')

In [21]:
#LiH
OB = np.load('../data/integrals/STO-3G/STO-3G_LiH_OB_d1-548_eq1.npy')
TB = np.load('../data/integrals/STO-3G/STO-3G_LiH_TB_d1-548_eq1.npy')

In [17]:
TB.shape

(2, 2, 2, 2)

In [ ]:
#N2
OB_n2 = np.load('../data/integrals/STO-3G/STO-3G_N2_OB_d1-19_eq1.npy')
TB_n2 = np.load('../data/integrals/STO-3G/STO-3G_N2_TB_d1-19_eq1.npy')

In [ ]:
#C2
OB = np.load('../data/integrals/STO-3G/STO-3G_C2_OB_d1-26_eq1.npy')
TB = np.load('../data/integrals/STO-3G/STO-3G_C2_TB_d1-26_eq1.npy')

In [23]:
FerOp = FermionicOperator(OB, TB)
#FerOp = FerOp.fermion_mode_elimination([9])
mapping = FerOp.mapping('jordan_wigner')
weights = [w[0] for w in mapping.paulis]
operators = [w[1].to_label() for w in mapping.paulis]
ha = nk.operator.PauliStrings(operators, weights)
#print(ha.to_sparse())
res = nk.exact.lanczos_ed(ha, k=1, compute_eigenvectors=False)
print("Exact ground state energy = {}".format(res))

Exact ground state energy = [-6.29466025]


In [20]:
P_op = ['IIII', 'ZZII', 'ZIZI', 'IZZI', 'ZIIZ', 'IZIZ', 'IIZZ', 'ZZZZ']
P_w = [3/8, -1/8, -1/8, -1/8, -1/8, -1/8, -1/8, 3/8]
P = nk.operator.PauliStrings(P_op, P_w)
print(P.to_sparse())

  (3, 3)	(1+0j)
  (5, 5)	(1+0j)
  (6, 6)	(1+0j)
  (9, 9)	(1+0j)
  (10, 10)	(1+0j)
  (12, 12)	(1+0j)


In [14]:
P.to_dense().conjugate().transpose() @ ha.to_dense() @ P.to_dense()

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 64 is different from 16)

In [ ]:
for perm in itertools.permutations('ijkl'):
    #H2
    OB = np.load('../data/integrals/STO-3G/STO-3G_H2_OB_d0-7339_eq1.npy')
    TB = np.load('../data/integrals/STO-3G/STO-3G_H2_TB_d0-7339_eq1.npy')
    TB = np.einsum('ijkl->' + ''.join(perm), TB)
    FerOp = FermionicOperator(OB, TB)
    weights = [w[0].real for w in FerOp.mapping('jordan_wigner').paulis]
    operators = [w[1].to_label() for w in FerOp.mapping('jordan_wigner').paulis]
    ha = nk.operator.PauliStrings(operators, weights)
    res = nk.exact.lanczos_ed(ha, k=2, compute_eigenvectors=False)
    print("Exact ground state energy (switch {})= {}".format(''.join(perm), res))